# Titanic_logistic_regression

**Date:** 26 January 2023

**Author:** Supawit Srisupap

In [102]:
install.packages("titanic")
library(titanic)
library(tidyverse)

Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done



In [103]:
glimpse(titanic_train)

Rows: 714
Columns: 12
$ PassengerId <int> 1, 2, 3, 4, 5, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 19…
$ Survived    <int> 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1…
$ Pclass      <int> 3, 1, 3, 1, 3, 1, 3, 3, 2, 3, 1, 3, 3, 3, 2, 3, 3, 2, 2, 3…
$ Name        <chr> "Braund, Mr. Owen Harris", "Cumings, Mrs. John Bradley (Fl…
$ Sex         <chr> "male", "female", "female", "female", "male", "male", "mal…
$ Age         <dbl> 22, 38, 26, 35, 35, 54, 2, 27, 14, 4, 58, 20, 39, 14, 55, …
$ SibSp       <int> 1, 1, 0, 1, 0, 0, 3, 0, 1, 1, 0, 0, 1, 0, 0, 4, 1, 0, 0, 0…
$ Parch       <int> 0, 0, 0, 0, 0, 0, 1, 2, 0, 1, 0, 0, 5, 0, 0, 1, 0, 0, 0, 0…
$ Ticket      <chr> "A/5 21171", "PC 17599", "STON/O2. 3101282", "113803", "37…
$ Fare        <dbl> 7.2500, 71.2833, 7.9250, 53.1000, 8.0500, 51.8625, 21.0750…
$ Cabin       <chr> "", "C85", "", "C123", "", "E46", "", "", "", "G6", "C103"…
$ Embarked    <chr> "S", "C", "S", "S", "S", "S", "S", "S", "C", "S", "S", "S"…


In [104]:
# drop na
titanic_train <- na.omit(titanic_train)

In [105]:
# split data
set.seed(53)
n <- nrow(titanic_train)
id <- sample(1:n, size= n*0.7) # 70% train , 30% test
train_data <- titanic_train[id, ]
test_data <- titanic_train[-id, ]

## Train model

In [106]:
# train data
log_model <- glm(Survived ~ Pclass + Sex + SibSp + Age, data = train_data, family = "binomial")
summary(log_model)


Call:
glm(formula = Survived ~ Pclass + Sex + SibSp + Age, family = "binomial", 
    data = train_data)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.7168  -0.6704  -0.3885   0.6118   2.4745  

Coefficients:
            Estimate Std. Error z value Pr(>|z|)    
(Intercept)  5.43434    0.65102   8.347  < 2e-16 ***
Pclass      -1.32484    0.16992  -7.797 6.34e-15 ***
Sexmale     -2.52040    0.25675  -9.817  < 2e-16 ***
SibSp       -0.35735    0.13569  -2.634  0.00845 ** 
Age         -0.04340    0.01007  -4.310 1.63e-05 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 665.95  on 498  degrees of freedom
Residual deviance: 447.28  on 494  degrees of freedom
AIC: 457.28

Number of Fisher Scoring iterations: 5


In [107]:
train_data$prob_sur <- predict(log_model, type="response")
train_data$pred_sur <- ifelse(train_data$prob_sur >= 0.5,1,0)
#train_data %>% 
#  select(PassengerId, Survived, prob_sur, pred_sur)

## Test model

In [108]:
test_data$prob_sur <- predict(log_model,newdata = test_data, type="response")
test_data$pred_sur <- ifelse(test_data$prob_sur >= 0.5,1,0)
#test_data %>% 
#  select(PassengerId, Survived, prob_sur, pred_sur)

## Accuracy

In [109]:
conmat <- table(train_data$Survived,train_data$pred_sur, dnn = c("Actual","Predicted"))
print(conmat)

      Predicted
Actual   0   1
     0 270  36
     1  56 137


In [110]:
acc_test <- (conmat[1,1]+conmat[2,2])/sum(conmat)
prec_test <- conmat[2,2]/(conmat[2,2]+conmat[2,1])
rec_test <- conmat[2,2]/(conmat[2,2]+conmat[1,2])
f1_test <- 2*((prec*rec)/(prec+rec))

In [111]:
cat("Test Model", "\nAccuracy:", acc_test, "\nPrecision:", prec_test, "\nRecall:", rec_test, "\nF1 Scorce:", f1_test)

Test Model 
Accuracy: 0.8156313 
Precision: 0.7098446 
Recall: 0.7919075 
F1 Scorce: 0.7486339